In [38]:
import numpy as np
import random
import warnings
from collections import Sequence
from itertools import repeat

In [39]:
# Crossover operations

__all__ = ['OnePoint_Crossover', 'TwoPoint_Crossover', 'Uniform_Crossover', 'Ordered_Crossover']

In [40]:
def OnePoint_Crossover(parent1, parent2):
    
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2


def TwoPoint_Crossover(parent1, parent2):
    size = min(len(parent1), len(parent2))
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    child1 = parent1[:cxpoint1] + parent2[cxpoint1:cxpoint2] + parent1[cxpoint2:]
    child2 = parent2[:cxpoint1] + parent1[cxpoint1:cxpoint2] + parent2[cxpoint2:]
    return child1, child2


def Uniform_Crossover(parent1, parent2, indpb = 0.5):
    size = min(len(parent1), len(parent2))
    child1, child2 = parent1[:], parent2[:]
    for i in range(size):
        if random.random() < indpb:
            child1[i], child2[i] = parent2[i], parent1[i]
    return child1, child2


def Ordered_Crossover(ind1, ind2):
   
    size = min(len(ind1), len(ind2))
    a, b = random.sample(range(size), 2)
    if a > b:
        a, b = b, a

    holes1, holes2 = [True] * size, [True] * size
    for i in range(size):
        if i < a or i > b:
            holes1[ind2[i]] = False
            holes2[ind1[i]] = False

    # We must keep the original values somewhere before scrambling everything
    temp1, temp2 = ind1, ind2
    k1, k2 = b + 1, b + 1
    for i in range(size):
        if not holes1[temp1[(i + b + 1) % size]]:
            ind1[k1 % size] = temp1[(i + b + 1) % size]
            k1 += 1

        if not holes2[temp2[(i + b + 1) % size]]:
            ind2[k2 % size] = temp2[(i + b + 1) % size]
            k2 += 1

    # Swap the content between a and b (included)
    for i in range(a, b + 1):
        ind1[i], ind2[i] = ind2[i], ind1[i]

    return ind1, ind2


In [41]:
# Mutation operations

__all__ = [ 'ShuffleIndexes_Mutation', 'FlipBit_Mutation', 'Inversion_Mutation']

In [42]:
def ShuffleIndexes_Mutation(individual, indpb = 0.1):

    size = len(individual)
    for i in range(size):
        if random.random() < indpb:
            swap_indx = random.randint(0, size - 2)
            if swap_indx >= i:
                swap_indx += 1
            individual[i], individual[swap_indx] = \
                individual[swap_indx], individual[i]

    return individual,



def FlipBit_Mutation(individual, indpb=0.1):
    for i in range(len(individual)):
        if random.random() < indpb:
            individual[i] = int(not individual[i])
    return individual,


def Inversion_Mutation(individual):

    size = len(individual)
    if size == 0:
        return individual,

    index_one = random.randrange(size)
    index_two = random.randrange(size)
    start_index = min(index_one, index_two)
    end_index = max(index_one, index_two)

    # Reverse the contents of the individual between the indices
    individual[start_index:end_index] = individual[start_index:end_index][::-1]

    return individual,


In [43]:
# Selection operations

__all__ = ['Random_Selection', 'Best_Selection', 'RouletteWheel_Selection', 'Tournament_Selection']

In [44]:
def Random_Selection(individuals, k):
    return [random.choice(individuals) for _ in range(k)]


def Elitism_Selection(individuals, k):
    return sorted(individuals, key=calculate_fitness, reverse=True)[:k]



def Tournament_Selection(individuals, k, tournsize=3):
    chosen = []
    for _ in range(k):
        aspirants = Random_Selection(individuals, tournsize)
        chosen.append(max(aspirants, key=calculate_fitness))
    return chosen

def Tournament_Selection_TSP_MIN(individuals, k, tournsize=3):
    chosen = []
    for _ in range(k):
        aspirants = random.sample(individuals, tournsize)
        chosen.append(min(aspirants, key=calculate_fitness))
    return chosen


def RouletteWheel_Selection(individuals, k):
    fitness_values = [calculate_fitness(individual) for individual in individuals]
    sum_fits = sum(fitness_values)
    chosen = []
    for _ in range(k):
        u = random.random() * sum_fits
        sum_ = 0
        for i, individual in enumerate(individuals):
            sum_ += fitness_values[i]
            if sum_ > u:
                chosen.append(individual)
                break
    return chosen
